<a href="https://colab.research.google.com/github/fgprevito/building-intuition/blob/main/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OBJECTIVE:

- Figure out how will we can explain how a company perofrms just by knowing company, sector, and year data.
- Figure out what cross validation means in this context. What does it look like? How would that play out?
- When we have a regression model, we'll read the outliers.

Eventually we will try to get to a point where we need a LLM to interpret the results.


CONTEXT:

*"There are few things which we know which aren't capable of being reduced to a mathematical reasoning. And when they cannot, it is a sign that our knowldge of them is very small and confused. Where a mathematic reasoning can be had, it is a great folly to make use of any other type of reasoning, as that would be like groping for a thing in the dark when you have a candle standing by you."*

**Start with reverse engineering the data**

I want to start by trying to solve for the following... There are 2 realities:

  1: the model I have in mind makes sense

  2: it doesn't

Now, note that I'm not interested in getting into a philosophical debate about the linearity of companies. Let's just suppose a linear model can actually can explain data well. How do I know I'm actually explaining data well and not just fooling myself into these factors' relevancy? How do I tell difference beteen good explanations using models and bad explanations using models? Easiest way to do this is using 2 kinds of fake data:

  1: Fake data that uses my model to make fake data

  2: Fake data that obviously violates the assumptions of my model

There are 2 parts to the model assumptions we've made:

  (1) there are factors (companies belong in the secotrs, companies affected by year theyre in, etc).

  (2) There's a small set of factors that added together, form estimates on how a company fairs.

Fake data make most sense where we make up values for these different factors and add them up. Suppose we have factors that added up like this, does my model recover them? That's a sanity check. Reality for operation of companies is theyre extremely nonlinear due to inefficiencies (department rivalries, acquisitons). A human CEO will attempt as much as possible to atempt additive mentality though.

    NOTE: Trying to learn math here!
      If someone asks you for incredibly complicated model for stock prices you know they're morons
      Do the assumptions of my model make sense with reality?
      CFO enforces accounting view of additive cash flows to rationalize first order approximate answers
      Long term successs wont be determined by this mentality though

The other set of data need to totally violate my model assumptions (AKA make the data not related to each other). Ex.: truly independent random numbers. In this case my additive model should tell me I dont see the effect. I should see numbers that tell me there's no effect between my presumed causal mechanism and the metric I wish to optimize or phenomena I am attempting to understand.
  
NOTE: Your job isnt to find new variables, its to eliminate as many variables as you can from contention. This is the key to being a good quant: having an extremely disciplined process where you can eliminate irrelevant economic variables from contention

So my objective here today is:

- make fake data that actually models different things we've discussed

  (1) what company am i

  (2) what year is this

  (3) sector

  (4) outcome = what company am i in, what year am i in, sector

  - how many comapny, year, sector factors do i have?

    - make design matrix for that

    - choose numbers for that
    
    - outcomes will be sums for this

IMPORTANT:
 If I make data that are generated from an honest to God model, under what assumptions does my model recapitulate that structure? We should realize there are specific conditions where we can recapitulate that data. On what conditions can you invert the covariance matrix? What does it mean for matrix to be non singular? We want intuition for this. If data is truly driven from an additive process there are only certain conditions where these will be recovered

Know how to make data to truly understand how to manipulate the processes (do i actually know what I think I'm doing? Even though fake data wont model reality exactly its good to know what parts of reality I cant control (Ex. are my models sensitive to one time deviations to the way things work?)

Now, why did we pick pharma to start? Because people forget this a lot. If you think you are smart you should be doing something that on the margin creates additonal cash productivity, better mgmt, etc. You need to do something more than everyone else. I have a suspicion that pharma as an industry has been flatlined on return on research. It would be interesting if when we adjust for year-wise & macroeconomic affects, the industry is flatlined. Means we've lost imagination and need to think different. All goes back to rationalizng with fake data that we know how this simple world works


In [1]:
# importing basic libraries

import pandas as pd
import numpy as np

Let's start by setting some constant variables. It's important that models
 should be DYNAMIC and easy for the end user to interact with and stress test.
 Anyone that gives you a complicated model with no way to easily stress test
 it is a moron.

In [11]:
# Constants

STOCK_PRICE_MAX = 100
NOISE_SCALE = 5

Created a list of companies and then created a data dictionary that
 designated an integer value for each of my companies
 we did this so that I can run the numbers through my design matrix

In [12]:
Company = ['Apple', 'Tesla', 'Microsoft', 'Amazon', 'Avronna', 'Novartis', 'Merck', 'Shell', 'Exxon', 'Chevron']
co_map = {x:ii for (ii,x) in enumerate(Company)}
print(co_map)

{'Apple': 0, 'Tesla': 1, 'Microsoft': 2, 'Amazon': 3, 'Avronna': 4, 'Novartis': 5, 'Merck': 6, 'Shell': 7, 'Exxon': 8, 'Chevron': 9}


Created an empty list and called it acc
 for a range of years, we will select one company per each year
 selected co will be 1 non selected co for that year will remain 0

In [13]:
acc = []

for yy in range(2008, 2008+5):
  for c in Company:
    new_row = np.zeros((len(Company), ))
    new_row[co_map[c]] = 1
    acc.append(new_row)

 np.vstack will stack our arrays we've just written vertically.

- basically, with our for loop, within the range of years we selected (number of times we want to reiterate through each company) we ran another for loop that iterates through each company in our comapny list
- first with our company we create a list of 0s the length of the number of companies in our list
- when we reach the company we currently are iterating through, we set it = to 1
- we append that row to our empty list 'acc' and then go to the next company in the same year
- once we iterate through all teh companies we move on to the next year and do the same

We should expect, for 10 companies over 5 years, that we get an array that has 50 rows and 10 columns. Our number of columns was set to a length equalling the number of companies, since after we iterate through 10 companies in 1 year we then move on to the next and go through the 10 companies again

Since we iterate through 10 companies 5 times, we should end up with 50 rows, 5 years of data for each of our 10 companies

In [15]:
design = np.vstack(acc)
design

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 

In [17]:
# checking shape of design just to make sure

design.shape

(50, 10)

Now we want to create our coefficients (weights). This is how our predictor variables will relate to our result variables. We can multiply it by stock price max so we could get some more realistic values between 0 and 100.

In [18]:
# fake_weights = np.random.rand(design.shape[1]) * STOCK_PRICE_MAX
fake_weights = np.random.rand(design.shape[1])

Questions to consider now: What are our fake weights? Why did we get these weights? where did they come from? How were they generated?

In [21]:
fake_weights

array([0.28992593, 0.69090884, 0.44452577, 0.64322638, 0.38844958,
       0.46328897, 0.32833659, 0.69038083, 0.86734067, 0.44231478])

Now we take the dot product of our design matrix by our fake_weights to give the predicted values of our response var for each observations

In [22]:
fake_outcomes = np.dot(design, fake_weights)

Generating random data for me to predict here.

Right now the data is generated from 0-1. In order to make the data more symmetrical we subtract .5 so it goes from -.5 to .5

The noise is our error term, increasing variance and unpredictability

In [24]:
# fake_outcomes += (np.random.rand(fake_outcomes.shape[0]) - 0.5 ) * NOISE_SCALE
fake_outcomes += np.random.rand(fake_outcomes.shape[0])

In [25]:
np.random.rand(fake_outcomes.shape[0])

array([0.54808492, 0.64958869, 0.87584047, 0.00098975, 0.59399307,
       0.43443394, 0.25699806, 0.14373301, 0.67525752, 0.93749254,
       0.5372618 , 0.7046891 , 0.31172198, 0.46741913, 0.87989553,
       0.81501497, 0.96660296, 0.4303964 , 0.55326329, 0.17446244,
       0.69354458, 0.03627493, 0.92620906, 0.17228107, 0.00565652,
       0.94818161, 0.69949637, 0.19891525, 0.16867032, 0.25251965,
       0.06197036, 0.89146483, 0.1230127 , 0.47883486, 0.75677126,
       0.20076339, 0.10294996, 0.32155852, 0.73099331, 0.35757569,
       0.71235392, 0.26979955, 0.78102174, 0.77650829, 0.58259371,
       0.00945105, 0.08444184, 0.82748878, 0.49025149, 0.9618718 ])

In [26]:
fake_outcomes

array([1.4490108 , 1.75880293, 0.82436732, 1.81095628, 1.04873819,
       1.25513262, 2.01350524, 1.61032802, 1.91150101, 2.23715016,
       1.39193599, 1.95790215, 0.95906701, 1.50661333, 1.97291801,
       1.2613812 , 0.68900081, 1.16367761, 1.66390574, 1.31595947,
       0.53705748, 1.27889843, 1.75747066, 2.2185214 , 1.47239134,
       1.0924999 , 1.44801788, 2.24347254, 1.88727146, 1.89973669,
       1.20052488, 1.08671895, 1.91196315, 1.43211632, 1.74205744,
       1.6017717 , 0.87649482, 1.45273695, 1.85312556, 1.55638486,
       1.57164858, 1.56391411, 1.74890826, 1.07879779, 1.45205429,
       1.56635566, 2.16957958, 1.82202852, 1.98337291, 1.3422022 ])

In [27]:
# this gives us the fake outcome for every tenth value in our matrix (Apple||0)

fake_outcomes[::len(Company)]

array([1.4490108 , 1.39193599, 0.53705748, 1.20052488, 1.57164858])

In [28]:
# What is the average of the predicted values for y when company is Apple?

np.mean(fake_outcomes[::len(Company)])

1.2300355454929357

Using linear reg library for now but will do it manually later

In [29]:
from sklearn.linear_model import LinearRegression

In [30]:
mylr = LinearRegression()

In [31]:
mylr.fit(design, fake_outcomes)

LinearRegression()

In [32]:
fake_outcomes.shape

(50,)

In [34]:
mylr.coef_

array([-0.30296342, -0.00375165, -0.09264368,  0.07640206,  0.00463289,
       -0.17757075, -0.0936793 ,  0.12544976,  0.32683637,  0.13728771])

In [35]:
mylr.intercept_

1.5329989640440693

In [36]:
mylr.coef_ + mylr.intercept_

array([1.23003555, 1.52924732, 1.44035528, 1.60940102, 1.53763185,
       1.35542821, 1.43931967, 1.65844873, 1.85983534, 1.67028668])

What we just did:
- we had repeated obs for each company a number of times
- we perfectly modeled the world as something evident
- there was only one factor called company, no random noise
- what linear regression does is subtract average value of outcome and fits intercept to that
- intercept is something that takes care of "how much do i get for free?"

In [37]:
mylr = LinearRegression(fit_intercept=False)

mylr.fit(design, fake_outcomes)

mylr.coef_

array([1.23003555, 1.52924732, 1.44035528, 1.60940102, 1.53763185,
       1.35542821, 1.43931967, 1.65844873, 1.85983534, 1.67028668])

Interecept is 0 because we said dont fit it
- dont fit intercept means dont pretend you get values for free


In [38]:
mylr.coef_ - fake_weights

array([0.94010961, 0.83833847, 0.99582952, 0.96617464, 1.14918228,
       0.89213924, 1.11098308, 0.9680679 , 0.99249467, 1.2279719 ])

In [39]:
fake_outcomes.reshape(5, len(Company))

array([[1.4490108 , 1.75880293, 0.82436732, 1.81095628, 1.04873819,
        1.25513262, 2.01350524, 1.61032802, 1.91150101, 2.23715016],
       [1.39193599, 1.95790215, 0.95906701, 1.50661333, 1.97291801,
        1.2613812 , 0.68900081, 1.16367761, 1.66390574, 1.31595947],
       [0.53705748, 1.27889843, 1.75747066, 2.2185214 , 1.47239134,
        1.0924999 , 1.44801788, 2.24347254, 1.88727146, 1.89973669],
       [1.20052488, 1.08671895, 1.91196315, 1.43211632, 1.74205744,
        1.6017717 , 0.87649482, 1.45273695, 1.85312556, 1.55638486],
       [1.57164858, 1.56391411, 1.74890826, 1.07879779, 1.45205429,
        1.56635566, 2.16957958, 1.82202852, 1.98337291, 1.3422022 ]])

In [40]:
np.mean(fake_outcomes.reshape(5, len(Company)), axis = 0)

array([1.23003555, 1.52924732, 1.44035528, 1.60940102, 1.53763185,
       1.35542821, 1.43931967, 1.65844873, 1.85983534, 1.67028668])

Question: Why are these numbers the same as my linear regression coef?

Next: we will make copy of this and add a year factor
- years will be shared for everyone (2008 is same 2008 for everyone)